# **PRACTICA 2**

## **DETERMINACIÓN DE TIPOS DE ESTRELLAS**

 1. Programar k-means, utilizando buenas prácticas de programación, es decir, debe ser una función y comparar los resultados y la eficiencia con la implementación de k-means en scikit-learn

2. Tenemos variables categóricas (Color, Clase Espectral). Tenemos dos
posibilidades: (a) codificar con one-hot-encoding; (b) codificar como una variable
ordinal. Tener en cuenta que el color está asociado a la cantidad de energía, y
algo parecido con la clase espectral.

3. Aplicar al menos dos diferentes algoritmos de clustering de los explicados en clase (K-Means, Hierarchical Clustering/Dendrogramas, DBSCAN), comparando y discutiendo los resultados que se obtienen de ellos

4. Discutir los resultados que se obtienen si a los atributos categóricos se les aplica one-hot-encoding o se asigna un valor numérico a las secuencias (variable ordinal).


5. A partir de los resultados obtenidos, ¿qué pipeline de clustering, es decir, qué transformaciones de datos, algoritmo, con sus hiperparámetros, transformación de datos y análisis de resultados recomendaría realizar?

6. ¿Hay similitudes con los grupos obtenidos en el punto 4? Explicar